In [152]:
url='https://www.freelancer.com'
access_token='M6X6wTeZ4JfyUgGJA5VrDDX5VflEpG'

In [153]:
import requests
import re
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
from selenium import webdriver
import time
import os
import urllib.parse

In [154]:
# driver = webdriver.Chrome(executable_path = "chromedriver-win64/chromedriver.exe")

# url = "https://www.freelancer.com/job/"

# driver.get(url)
# time.sleep(5)
# # Lấy HTML sau khi trang đã tải xong
# html_content = driver.page_source
# # Phân tích HTML bằng BeautifulSoup
# # soup = BeautifulSoup(html_content, 'html.parser')

# driver.quit()
# file_path = "C:/Users/Admin/OneDrive - VNU-HCMUS/Documents/K1N4/UD_PTDLTM/freelancer_jobs.html"

# with open(file_path, "w", encoding="utf-8") as file:
#     file.write(html_content)

In [155]:
# # 1 = web, 2 = Mobile, 6 = Data,  7 = AI


# #  Categories to target
# categories = [2, 3, 6, 7]

# # Create a dictionary to hold lists of tuples (href, job_count) for each category
# category_links = {category_id: [] for category_id in categories}

# # Parse the HTML
# soup = BeautifulSoup(html_content, 'html.parser')

# # Iterate over the categories
# for category_id in categories:
#     category_section = soup.find('section', id=f'category-{category_id}')
#     if category_section:
#         # Find all list items (jobs) in the category
#         job_items = category_section.find_all('li')
        
#         for job_item in job_items:
#             # Find the link inside the job item
#             job_link = job_item.find('a', class_='PageJob-category-link')
#             if job_link:
#                 # Extract the job count using regex
#                 job_text = job_link.get_text(strip=True)
#                 job_count = re.search(r'\((\d+)\)', job_text)
                
#                 if job_count and int(job_count.group(1)) > 0:
#                     job_href = job_link['href']
#                     # Save href and job_count as a tuple (href, job_count)
#                     category_links[category_id].append((job_href, int(job_count.group(1))))

# # Print the lists for each category
# for category_id, href_list in  category_links.items():
#     print(f'Category {category_id} links: {href_list}')


In [210]:
#url="https://www.freelancer.com/jobs/pytorch/?page=1"
def get_job_atrributes(url):
    soup=BeautifulSoup(requests.get(url).content, 'html.parser')

    # Get the seo_url through class="JobSearchCard-primary-heading" html
    anchor_tag = soup.find_all('a', class_='JobSearchCard-primary-heading-link')

    # Get the 'href' attribute
    href_list = [tag['href'] for tag in anchor_tag]

    return href_list

def read_json_list_jobs(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)
    
    
def get_seo_url_json_list_jobs(list_url):
    contests_seo_urls=[]
    projects_seo_urls=[]
    for url in list_url:
        
        seo_urls=get_job_atrributes(url)
        for seo_url in seo_urls:
            
            #If have 'contest' in url, append to contest list
            if seo_url[0:9]=='/contest/':
                contests_seo_urls.append(seo_url)
            else :
                projects_seo_urls.append(seo_url)
                
    #Remove "/projects/" in url
    projects_seo_urls=[url.replace("/projects/","") for url in projects_seo_urls]
    
    #Apply regex to get the contest id contain 7 digits
    contests_id=[re.search(r'\d{7}',url).group(0) for url in contests_seo_urls]
    
    return projects_seo_urls, contests_id

In [211]:
def create_api_url_request(p_base_url,p_params):
    

    # Encode parameters
    encoded_params = urllib.parse.urlencode(p_params, doseq=True)

    # Construct the full URL
    url = f'{p_base_url}?{encoded_params}'

    # Headers for authorization
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    return url, headers

In [216]:
def get_contests(p_contests_id):
    
    # Parameters contructed as a dictionary
    params = {
        'contests[]': p_contests_id,
        'job_details':True,
    }
    base_url = 'https://www.freelancer.com/api/contests/0.1/contests/'
    url, headers = create_api_url_request(base_url, params)
    try: 
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error: {e}')
        return None
def get_projects(p_seo_urls):
    # Parameters contructed as a dictionary
    params = {
        'seo_urls[]': p_seo_urls,
        'full_description': True,
        'job_details': True,
    }
    base_url = 'https://www.freelancer.com/api/projects/0.1/projects/'
    url, headers = create_api_url_request(base_url, params)
    try: 
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error: {e}')
        return None

In [220]:
def get_data_jobs(p_category,p_min,p_max):
    current_file_path=os.getcwd()
    #json.loads(open(, "r", encoding="utf-8").read())
    file_path="/".join((os.path.dirname(os.path.dirname(os.path.dirname(current_file_path))),\
                        "data","utils","list_jobs_URL_of_each_category.json"))
    body_content=read_json_list_jobs(file_path)


    #Get 10 url of first category
    body_content=body_content[str(p_category)][p_min:p_max]

    projects_seo_urls, contests_id=get_seo_url_json_list_jobs(body_content)

    projects=None
    contests=None
    if len(projects_seo_urls)>0:
        try:
            projects=get_projects(projects_seo_urls)
        except:
            pass
    if len(contests_id)>0:
        try:
            contests=get_contests(contests_id)
        except:
            pass
    return projects, contests


In [218]:
projects,contests=get_data_jobs(2,10,20)

In [257]:
def to_df_projects_contests(projects,contests):
    df=pd.DataFrame(columns=["job_id","owner_id","type","title","services","skills","description",
                             "date_posted","time_remaining","budget_min","budget_max","budget_currency",
                             "exchange_rate","bid_count","average_bid","status","language","url"])
    #Save projects to dataframe
    for project in projects['result']['projects']:
        job_id=project['id']
        owner_id=project['owner_id']
        _type="Project"
        title=project['title']
        services=project['seo_url'].split("/")[0]
        skills=[skill['name'] for skill in project['jobs']]
        description=project['description']
        date_posted=project['submitdate']
        time_free_bids_expire=project['time_free_bids_expire']
        budget_min=project['budget']['minimum']
        budget_max=project['budget']['maximum']
        budget_currency=project['currency']['code']
        exchange_rate=project['currency']['exchange_rate']
        bid_count=project['bid_stats']['bid_count']
        average_bid=project['bid_stats']['bid_avg']
        status=project['status']
        language=project['language']
        url="https://www.freelancer.com/projects"+project['seo_url']
        #Check if the job_id already exists in the dataframe
        if job_id in df['job_id'].values:
            continue
        else:
            new_row_df=pd.DataFrame({"job_id":job_id,"owner_id":owner_id,"type":_type,"title":title,\
                                    "services":services,"skills":[skills],"description":description,\
                                    "date_posted":date_posted,"time_remaining":time_free_bids_expire,\
                                    "budget_min":budget_min,"budget_max":budget_max,"budget_currency":budget_currency,\
                                    "exchange_rate":exchange_rate,"bid_count":bid_count,"average_bid":average_bid,\
                                    "status":status,"language":language,"url":url})
            df=pd.concat([df,new_row_df],ignore_index=True)
            
    #Save contests to dataframe
    for contest in contests['result']['contests']:
        job_id=contest['id']
        owner_id=contest['owner_id']
        _type="Contest"
        title=contest['title']
        services=None
        skills=[skill['name'] for skill in contest['jobs']]
        description=contest['description']
        date_posted=contest['time_submitted']
        time_remaining=None
        budget_min=contest['prize']
        budget_max=contest['prize']
        budget_currency=contest['currency']['code']
        exchange_rate=contest['currency']['exchange_rate']
        bid_count=None
        average_bid=None
        status=contest['status']
        language=contest['language']
        url="https://www.freelancer.com/"+contest['seo_url']
        #Check if the job_id already exists in the dataframe
        if job_id in df['job_id'].values:
            continue
        else:
            new_row_df=pd.DataFrame({"job_id":job_id,"owner_id":owner_id,"type":_type,"title":title,\
                                    "services":services,"skills":[skills],"description":description,\
                                    "date_posted":date_posted,"time_remaining":time_remaining,\
                                    "budget_min":budget_min,"budget_max":budget_max,"budget_currency":budget_currency,\
                                    "exchange_rate":exchange_rate,"bid_count":bid_count,"average_bid":average_bid,\
                                    "status":status,"language":language,"url":url})
            df=pd.concat([df,new_row_df],ignore_index=True)
    return df


In [259]:
def save_projects_contests_to_csv(df,category,p_min,p_max):
    current_file_path=os.getcwd()
    file_path="/".join((os.path.dirname(os.path.dirname(os.path.dirname(current_file_path))),\
                        "data","raw","freelancer_jobs",f"freelancer_jobs_category_{category}_{p_min}_{p_max}.csv"))
    df.to_csv(file_path,index=False)
    return file_path

In [262]:
file_path=save_projects_contests_to_csv(df,2,10,20)

In [263]:
def get_projects_contests_id(file_path):
    df=pd.read_csv(file_path)
    return df[['job_id','type']].values

array([[38713328, 'Project'],
       [38712591, 'Project'],
       [38711748, 'Project'],
       [38711223, 'Project'],
       [38710274, 'Project'],
       [38708928, 'Project'],
       [38708739, 'Project'],
       [38708523, 'Project'],
       [38708442, 'Project'],
       [38708103, 'Project'],
       [38707774, 'Project'],
       [38707625, 'Project'],
       [38707328, 'Project'],
       [38706922, 'Project'],
       [38705981, 'Project'],
       [38705592, 'Project'],
       [38704879, 'Project'],
       [38704558, 'Project'],
       [38675241, 'Project'],
       [38703520, 'Project'],
       [38702891, 'Project'],
       [38702617, 'Project'],
       [38701246, 'Project'],
       [38700989, 'Project'],
       [38700487, 'Project'],
       [38699261, 'Project'],
       [38698983, 'Project'],
       [38698064, 'Project'],
       [38697915, 'Project'],
       [38697845, 'Project'],
       [38697586, 'Project'],
       [38697420, 'Project'],
       [38696959, 'Project'],
       [38

In [266]:
def get_bid(project_id):
    # Parameters contructed as a dictionary
    params = {
        'user_details':True,
    }
    base_url = 'https://www.freelancer.com/api/projects/0.1/projects/{}/bids/'.format(project_id)
    url, headers = create_api_url_request(base_url, params)
    try: 
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error: {e}')
        return None
def get_entrants(contest_id):
    # Parameters contructed as a dictionary
    params = {
        #'user_details':True,
    }
    base_url = 'https://www.freelancer.com/api/contests/0.1/contests/{}/entrants/'.format(contest_id)
    url, headers = create_api_url_request(base_url, params)
    try: 
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error: {e}')
        return None

In [267]:
data=get_entrants(2464255)

Error: 404 Client Error: Not Found for url: https://www.freelancer.com/api/contests/0.1/contests/2464255/entrants/?user_details=True


In [177]:
data

{'status': 'success',
 'result': {'bids': [{'id': 432635549,
    'bidder_id': 11081971,
    'project_id': 38694303,
    'retracted': False,
    'amount': 750.0,
    'period': 12,
    'description': "Hey there,\n\nI totally get what you're looking for. With a deep understanding of digital security and code signing certificates, I can help you secure those EV or OV certificates from trusted providers like Certum. My experience with Windows platforms and code signing ensures a smooth process. Plus, I'm all about meeting those weekly timelines and making sure you get what you need when you need it.\n\nCould you share more about the specific Windows versions or environments you’re working with?\n\nDo you have any preferences regarding the providers besides Certum?\n\nAre there any special considerations you have regarding the certificates' deployment?\n\nLooking forward to working together!\n\nBest,  \nMuhammad Awais",
    'project_owner_id': 79898140,
    'submitdate': 1729238982,
    'buy